In [138]:
def pwm(lista):
    nuc = ['A', 'C', 'G', 'T']
    profile=[[0 for i in range(len(lista[0]))] for j in range(len(nuc))]
    for instance in lista:
        for j in range(len(instance)):
            residue=instance[j]
            profile[nuc.index(residue)][j]+=1
            profile[nuc.index(residue)][j]=float(profile[nuc.index(residue)][j])
    import numpy as np
    pwm = np.array(profile)
    pwm = pwm/len(lista)
    return(pwm)

In [139]:
import regex as re

def kmers(genomefile, k):
    kmertable = {} 
    for i in range(len(seq)-k):
        DNA=seq[i:i+k]
        if DNA not in kmertable.keys():
            kmertable[DNA]=1
        else:
            kmertable[DNA]+=1

    kmertable = {k: float(v) / len(seq) for k, v in kmertable.items()}
    kmertable = dict(sorted(kmertable.items()))
    return(list(kmertable.values()))

In [140]:
def pssm(pwm, nucfreqs):
    import numpy as np
    import math
    pseudocount=0.01
    pssm=[[0 for i in range(len(pwm[0]))] for j in range(len(nucfreqs))]
    for i in range(len(nucfreqs)):
        pssm[i]=(np.array(pwm[i])+pseudocount)/nucfreqs[i]
    for i in range(len(pssm)):
        for k in range(len(pssm[0])):
            pssm[i][k]=math.log(pssm[i][k])/math.log(2)
    return(np.array(pssm))

In [141]:
def pssmSearch(pssm, sequence, threshold):
    nuc = ['A', 'C', 'G', 'T']
    hits = []
    instances = []
    x = []
    allscores = []
    maxPssm = 0
    for j in range(len(pssm[0])):
        maxPssm = maxPssm + max(pssm[:,j])
    
    for i in range(len(sequence)-len(pssm[0])):
        x.append(i)
        instance=sequence[i:i+len(pssm[0])]
        score=0
        for l in range(len(instance)):
            score=score+pssm[nuc.index(instance[l])][l]
        if (score > threshold*maxPssm):
            hits.append(i)
            instances.append(instance)
            allscores.append(score)
    
    return(instances)

In [142]:
def pwmEntropyInformation(pwm):
    import numpy as np
    k = pwm.shape[1]

    information = np.zeros([1,k])
    for i in range(k):
        information[0,i] = 2-abs(sum([elem*np.log2(elem) for elem in pwm[:,i] if elem > 0]))
    
    sumInfo = np.sum(information)
    scaledSumInfo = sumInfo/k
    
    return(scaledSumInfo)

In [146]:
import regex as re
import random

for k in range(3,8):
    fn = "motifs_in_sequence.fa"
    
    #Make a list with all the 50 sequences
    seq_list = []
    with open (fn, "r") as file:
        for line in file:
            seq_list.append(line)
    
    #Make a string that contains all the sequences
    seq = ""
    for string in seq_list:
        seq+=string[:-1] #until -1 to ignore the '\n'
    
    #Find the frequences of each nucleotide in whole concatenated sequence
    nucfreqs = kmers(fn,1)
    
    I=0
    while I < 1.8:
        #Make a list with random kmers of length k
        random_kmers=[]
        for i in seq_list[:50]:
            seqs_kmers=[]
            for j in range(len(i)-k):
                seqs_kmers.append((i[j:j+k]))
            random_kmers.append(random.choice(seqs_kmers))

        mypwm=pwm(random_kmers)
        mypssm=pssm(mypwm, nucfreqs)
        pssmsearches = pssmSearch(mypssm, seq, 0.95)
        I=pwmEntropyInformation(pwm(pssmsearches))
        print(I)

        if I >= 1.8:
            print (f"k =",k)
            print(pssmsearches[0])
            print(mypwm, '\n')

2.0
k = 3
CAA
[[0.16 0.4  0.28]
 [0.34 0.18 0.24]
 [0.2  0.18 0.26]
 [0.3  0.24 0.22]] 

2.0
k = 4
TATC
[[0.22 0.34 0.2  0.28]
 [0.26 0.2  0.1  0.3 ]
 [0.2  0.22 0.32 0.18]
 [0.32 0.24 0.38 0.24]] 

1.6235250802274563
1.6143684940767085
2.0
k = 5
TACGA
[[0.14 0.34 0.28 0.2  0.34]
 [0.24 0.16 0.34 0.22 0.24]
 [0.26 0.28 0.24 0.34 0.3 ]
 [0.36 0.22 0.14 0.24 0.12]] 

1.6818553323583447
1.8469506943242517
k = 6
GTGTCT
[[0.24 0.28 0.26 0.2  0.28 0.12]
 [0.2  0.18 0.14 0.28 0.38 0.3 ]
 [0.36 0.22 0.36 0.18 0.12 0.26]
 [0.2  0.32 0.24 0.34 0.22 0.32]] 

1.8571428571428572
k = 7
ATCGTTG
[[0.44 0.24 0.16 0.22 0.26 0.18 0.22]
 [0.2  0.2  0.28 0.14 0.24 0.22 0.28]
 [0.26 0.26 0.28 0.32 0.18 0.26 0.28]
 [0.1  0.3  0.28 0.32 0.32 0.34 0.22]] 

